In [19]:
import os, sys
import glob
import math
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser

#Covarage matrix as csv file without qoutes, seperated by commas
folder = "/home/pato/Skrivebord/HPV_subtyping/Results/"
topRunName = "Reanalyze_user_S5-0184-347-HPV_genotypering_Sara_Opsaet_10_A_1424_14112022"
indFile = "AnnotationIndividualFiles_"+topRunName+".txt"
sumFile = "AnnotationSummary_"+topRunName+".txt"

indFile = folder + topRunName + "/" + indFile
sumFile = folder + topRunName + "/" + sumFile

sublineagesFile = "/home/pato/Skrivebord/HPV_subtyping/References/ReferenceToSublineage.csv"


In [35]:
sumFile

'/home/pato/Skrivebord/HPV_subtyping/Results/Reanalyze_user_S5-0184-347-HPV_genotypering_Sara_Opsaet_10_A_1424_14112022/AnnotationSummary_Reanalyze_user_S5-0184-347-HPV_genotypering_Sara_Opsaet_10_A_1424_14112022.txt'

In [30]:
sumdf = pd.read_csv(sumFile, sep = "\t")
inddf = pd.read_csv(indFile, sep = "\t")
sublineages = pd.read_csv(sublineagesFile, sep = ",", header=None)

In [22]:
sublineages

,0,1,2,3,4
0,Alpha-10,HPV11,A,A1,M14119
1,Alpha-10,HPV11,A,A2,FN907962
2,Alpha-10,HPV11,A,A3,pending
3,Alpha-10,HPV11,A,A4,pending
4,Alpha-10,HPV11,B,B,pending
...,...,...,...,...,...
152,Alpha-5,HPV82,C,C3,KF436801
153,Alpha-5,HPV82,C,C4,KF436802
154,Alpha-5,HPV82,C,C5,KF436803
155,Alpha-7,HPV85,A,A1,AF131950


In [23]:
genes = list(sumdf['GENEID'].unique())
cols = ['Patient','Type','Reference matched','Sublineage']
cols.extend(genes)

In [24]:
for item in cols.copy():
    if "DoesNotExist" in item:
        cols.remove(item)

In [25]:
cols

['Patient',
 'Type',
 'Reference matched',
 'Sublineage',
 'E6',
 'gene-E7-138',
 'E4_splice',
 'gene-E2-139',
 'gene-E4-112',
 'gene-L1-141']

In [36]:
sumJson = []
for patient in inddf.columns:
    # Making json with col names
    ptjson = {}
    for col in cols:
        ptjson.update({
            col : ""
        })
    
    # Getting patient name, reference and sublineage
    spl = patient.split(":")
    ref = spl[1]
    sublineage = ""
    type = ""
    # Getting sublineage class and type from the reference id
    iteration = 0

    for i in sublineages[4]:
        if i in ref:
            sublineage = sublineages[3][iteration]
            type = sublineages[1][iteration]
    iteration +=1

    ptjson.update({
        'Patient' : spl[0],
        'Type' : type,
        'Reference matched' : spl[1],
        'Sublineage' : sublineage
    })

    # For each row, get nuc and aa changes and match to gene key
    for row in inddf[patient]:
        # This check fails if value is NA
        if row == row:
            splt = row.split(" ")
            # Try to split, because if the cell contains a message such as "NoE4File", the column should be skipped. 
            if len(splt) != 4:
                continue
            nuc = splt[0]
            aa = splt[1]
            gene = splt[2]
            ptjson.update({
                gene : [nuc,aa]
            })
    #jsonlist = [ptjson]
    sumJson.append(ptjson)
sumJson

[{'Patient': '24_Baseline.IonXpress_006',
  'Type': 'HPV11',
  'Reference matched': 'HQ537752_1',
  'Sublineage': 'A1',
  'E6': '',
  'gene-E7-138': ['c.744G>T', 'p.T57T'],
  'E4_splice': ['c.3550C>T', 'p.H66Y'],
  'gene-E2-139': ['c.3550C>T', 'p.C266C'],
  'gene-E4-112': ['c.3550C>T', 'p.A74V'],
  'gene-L1-141': ['c.6539G>A', 'p.M325I']},
 {'Patient': '111_Baseline.IonXpress_041',
  'Type': 'HPV11',
  'Reference matched': 'HQ644236_1',
  'Sublineage': 'A1',
  'E6': '',
  'gene-E7-138': '',
  'E4_splice': '',
  'gene-E2-139': '',
  'gene-E4-112': '',
  'gene-L1-141': ''},
 {'Patient': '16_Baseline.IonXpress_005',
  'Type': '',
  'Reference matched': 'HPV16_K02718_1_revised',
  'Sublineage': '',
  'E6': ['c.187A>G', 'p.L28L'],
  'gene-E7-138': '',
  'E4_splice': '',
  'gene-E2-139': '',
  'gene-E4-112': '',
  'gene-L1-141': ''}]

In [34]:
# Convert to dataframe and save to file
formDf = pd.DataFrame.from_dict(sumJson)
filename = folder+"/"+topRunName+"/AnnotationFormatted_"+topRunName+".txt"
formDf.to_csv(filename, index = False)